Reddit

In [1]:
import requests
import pandas as pd
import time

In [2]:
base_url = 'https://api.pushshift.io/'

submission_url = 'reddit/search/submission' 
comment_url = 'reddit/search/comment'

In [3]:
#  https://www.reddit.com/r/AskCulinary/
# https://www.reddit.com/r/running/

In [4]:
#subreddit = 'AskCulinary'
subreddit = 'running'


params = {
    'subreddit' : subreddit,
    'size' : 100
}

In [5]:
base_url + submission_url

'https://api.pushshift.io/reddit/search/submission'

In [6]:
res = requests.get (base_url + submission_url, params)

In [7]:
res.status_code

200

In [8]:
data = res.json()

posts = data['data']
len(posts)

100

In [9]:
posts[0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'ilritorno',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_15rvbr',
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1609274965,
 'domain': 'self.running',
 'full_link': 'https://www.reddit.com/r/running/comments/kmmmpc/five_stages_of_running/',
 'gildings': {},
 'id': 'kmmmpc',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': True,
 'is_video': False,
 'link_flair_background_color': '#007373',
 'link_flair_css_class': 'linkflair-article',
 'link_flair_richtext': [],
 'link_flair_template_id': '21f33906-6c17-11e5-86ae-0e96416492c1',
 'link_flair_text': 'Article',
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': F

In [133]:
posts_data = pd.DataFrame (posts)
#posts_data.head()

In [134]:
posts_df = posts_data[['subreddit', 'selftext', 'title', 'author', 'created_utc']].copy()
posts_df.head()

,subreddit,selftext,title,author,created_utc
0,running,It's almost universally accepted that runners ...,"For Race Day, Nose or Mouth Breathing?",Gunner56,1609273208
1,running,"Hi all! New to the subreddit, but not to runni...",Beginning 50-State Marathoner Looking For Race...,Prince-Char1ming,1609272830
2,running,Say suppose I ran about 50-70 miles in one day...,Running and muscle loss?,Difficult_Hour_4799,1609271273
3,running,I just got into running this summer and the mo...,Training plan or advice for half marathon?,Okhellohey,1609271017
4,running,[removed],Weightlifting and running,Anja1111111,1609270803


In [135]:
posts_df['created_date'] = pd.to_datetime(posts_df['created_utc'], unit='s')
#posts_df.loc[:,'created_date'] = pd.to_datetime(posts_df['created_utc'], unit='s')

In [136]:
posts_df.tail()

,subreddit,selftext,title,author,created_utc,created_date
95,running,Time to start running again. Took a week and ...,Preparing for a May Cowtown,RunningManinTx,1609198649,2020-12-28 23:37:29
96,running,"Hey hey, apologies if this has been answered, ...",How to stack my long runs with rest day/easy r...,shipoopi29,1609198275,2020-12-28 23:31:15
97,running,Weird power from inside\n\nOk I know this migh...,Weird power,Outrageous_Ad5805,1609196777,2020-12-28 23:06:17
98,running,I've been slowly returning to running after ph...,Strategies for avoiding injury/resurgence afte...,nightpussy,1609196108,2020-12-28 22:55:08
99,running,"Hi, so I decided to try the Superfeet Running ...","For overpronators, Do you use insoles? Do they...",mlx01,1609194028,2020-12-28 22:20:28


In [137]:
#[len(post) for post in posts_df['selftext']]

#convert created utc to readable format
# UTC_datetime_timestamp = float(UTC_datetime.strftime("%s"))
# local_datetime_converted = datetime.datetime.fromtimestamp(UTC_datetime_timestamp)


In [138]:
posts_df.iloc[len(posts_df)-1]['created_utc']

1609194028

In [139]:
# get more data
num_loops = 10

params = {
    'subreddit' : subreddit,
    'size' : 100,
    'before': ''
}

for index in range(num_loops):
    # Get timestamp of oldest current post in the dataframe
    oldest_utc = posts_df.iloc[len(posts_df)-1]['created_utc']
    
    params['before'] = oldest_utc
    
    res = requests.get (base_url + submission_url, params)
    print (f'Query: {submission_url} Params: {params} \n Response code: {res.status_code} ')
    
    if res.status_code == 200:
        # if valid response, concat to end of dataframe.
        temp_data = res.json()['data']   # get array of reddit posts (each being a dict)
        temp_data_df = pd.DataFrame (temp_data)  # convert reddit post dict to dataframe
        
        # only keep relevant columns
        temp_posts_df = temp_data_df[['subreddit', 'selftext', 'title', 'author', 'created_utc']].copy()
        temp_posts_df['created_date'] = pd.to_datetime(temp_data_df['created_utc'], unit='s')

        # concat to existing dataframe of prior responses
        posts_df = pd.concat([posts_df, temp_posts_df], ignore_index=True)
        
        # sleep one second so as to not overload reddit API
        time.sleep(1)
        



Query: reddit/search/submission Params: {'subreddit': 'running', 'size': 100, 'before': 1609194028} 
 Response code: 200 
Query: reddit/search/submission Params: {'subreddit': 'running', 'size': 100, 'before': 1609116160} 
 Response code: 200 
Query: reddit/search/submission Params: {'subreddit': 'running', 'size': 100, 'before': 1609049264} 
 Response code: 200 
Query: reddit/search/submission Params: {'subreddit': 'running', 'size': 100, 'before': 1608976735} 
 Response code: 200 
Query: reddit/search/submission Params: {'subreddit': 'running', 'size': 100, 'before': 1608837461} 
 Response code: 200 
Query: reddit/search/submission Params: {'subreddit': 'running', 'size': 100, 'before': 1608738802} 
 Response code: 200 
Query: reddit/search/submission Params: {'subreddit': 'running', 'size': 100, 'before': 1608655227} 
 Response code: 200 
Query: reddit/search/submission Params: {'subreddit': 'running', 'size': 100, 'before': 1608557951} 
 Response code: 200 
Query: reddit/search/sub

In [140]:
#posts_df.iloc[190:210, 0:20]
posts_df.tail()

,subreddit,selftext,title,author,created_utc,created_date
1095,running,[removed],Running teammate rivalry,Fastboy56138,1608251056,2020-12-18 00:24:16
1096,running,[removed],Real question: How do you run longer distances...,KeepPunkRockElite,1608250647,2020-12-18 00:17:27
1097,running,[removed],Anyone Else Get Wildly Different Results Using...,Accomplished-Classic,1608250104,2020-12-18 00:08:24
1098,running,[removed],Got back at running after 5 months 💪if u had a...,paul081,1608249362,2020-12-17 23:56:02
1099,running,[removed],Another little magical running moment: share y...,SnooConfections3930,1608249077,2020-12-17 23:51:17


In [141]:
#posts_df.reset_index(inplace=True)

In [142]:
posts_df.to_csv(f'./data/{ subreddit }.csv', index=False)